In [1]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.zip
!unzip "UCI HAR Dataset.zip"


--2021-09-30 09:50:34--  https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60999314 (58M) [application/x-httpd-php]
Saving to: ‘UCI HAR Dataset.zip’

UCI HAR Dataset.zip 100%[===================>]  58.17M  31.7MB/s    in 1.8s    

2021-09-30 09:50:38 (31.7 MB/s) - ‘UCI HAR Dataset.zip’ saved [60999314/60999314]

Archive:  UCI HAR Dataset.zip
   creating: UCI HAR Dataset/
  inflating: UCI HAR Dataset/.DS_Store  
   creating: __MACOSX/
   creating: __MACOSX/UCI HAR Dataset/
  inflating: __MACOSX/UCI HAR Dataset/._.DS_Store  
  inflating: UCI HAR Dataset/activity_labels.txt  
  inflating: __MACOSX/UCI HAR Dataset/._activity_labels.txt  
  inflating: UCI HAR Dataset/features.txt  
  inflating: __MACOSX/UCI HAR Dataset/._features.txt  
  infl

In [2]:
import os
import numpy as np
import tensorflow as tf

In [3]:
from tensorflow import reshape
from collections import OrderedDict

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input,Conv1D, BatchNormalization, MaxPool1D, Dropout, Flatten, Dense

from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from sklearn.metrics import accuracy_score

In [5]:
def format_data_x(datafile):
    x_data = None
    for item in datafile:
        item_data = np.loadtxt(item, dtype=np.float)
        if x_data is None:
            x_data = np.zeros((len(item_data), 1))
        x_data = np.hstack((x_data, item_data))
    x_data = x_data[:, 1:]
    print(x_data.shape)
    X = None
    for i in range(len(x_data)):
        row = np.asarray(x_data[i, :])
        row = row.reshape(9, 128).T
        if X is None:
            X = np.zeros((len(x_data), 128, 9))
        X[i] = row
    print(X.shape)
    return X


In [6]:
def format_data_y(datafile):
    data = np.loadtxt(datafile, dtype=np.int) - 1
    YY = np.eye(6,dtype=np.int)[data]
    return YY

In [7]:
def load_data():
    import os
    if os.path.isfile('data/data_har.npz') == True:
        data = np.load('data/data_har.npz')
        X_train = data['X_train']
        Y_train = data['Y_train']
        X_test = data['X_test']
        Y_test = data['Y_test']
    else:
        # This for processing the dataset from scratch
        # After downloading the dataset, put it to somewhere that str_folder can find
        str_folder = './UCI HAR Dataset/'
        INPUT_SIGNAL_TYPES = [
            "body_acc_x_",
            "body_acc_y_",
            "body_acc_z_",
            "body_gyro_x_",
            "body_gyro_y_",
            "body_gyro_z_",
            "total_acc_x_",
            "total_acc_y_",
            "total_acc_z_"
        ]

        str_train_files = [str_folder + 'train/' + 'Inertial Signals/' + item + 'train.txt' for item in
                           INPUT_SIGNAL_TYPES]
        str_test_files = [str_folder + 'test/' + 'Inertial Signals/' + item + 'test.txt' for item in INPUT_SIGNAL_TYPES]
        str_train_y = str_folder + 'train/y_train.txt'
        str_test_y = str_folder + 'test/y_test.txt'

        X_train = format_data_x(str_train_files)
        X_test = format_data_x(str_test_files)
        Y_train = format_data_y(str_train_y)
        Y_test = format_data_y(str_test_y)

    return X_train, Y_train, X_test, Y_test

In [8]:
X_train, Y_train, X_test, Y_test = load_data()

(7352, 1152)
(7352, 128, 9)
(2947, 1152)
(2947, 128, 9)


## Client as Data Shards

In [9]:
import random

In [10]:
def create_clients(image_list, label_list, num_clients=10, initial='clients'):

    #create a list of client names
    client_names = ['{}_{}'.format(initial, i+1) for i in range(num_clients)]

    #randomize the data
    data = list(zip(image_list, label_list))
    random.shuffle(data)

    #shard data and place at each client
    size = len(data)//num_clients
    shards = [data[i:i + size] for i in range(0, size*num_clients, size)]

    #number of clients must equal number of shards
    assert(len(shards) == len(client_names))

    return {client_names[i] : shards[i] for i in range(len(client_names))} 

In [11]:
#create clients
clients = create_clients(X_train, Y_train, num_clients=10, initial='client')

## Batching Data into tensorflow formats

In [12]:
def batch_data(data_shard, bs=32):
    '''Takes in a clients data shard and create a tfds object off it
    args:
        shard: a data, label constituting a client's data shard
        bs:batch size
    return:
        tfds object'''
    #seperate shard into data and labels lists
    data, label = zip(*data_shard)
    dataset = tf.data.Dataset.from_tensor_slices((list(data), list(label)))
    return dataset.shuffle(len(label)).batch(bs)

In [13]:
#process and batch the training data for each client
clients_batched = dict()
for (client_name, data) in clients.items():
    clients_batched[client_name] = batch_data(data)
    
#process and batch the test set  
test_batched = tf.data.Dataset.from_tensor_slices((X_test, Y_test)).batch(len(Y_test))

In [14]:
clients_batched

{'client_1': <BatchDataset shapes: ((None, 128, 9), (None, 6)), types: (tf.float64, tf.int64)>,
 'client_10': <BatchDataset shapes: ((None, 128, 9), (None, 6)), types: (tf.float64, tf.int64)>,
 'client_2': <BatchDataset shapes: ((None, 128, 9), (None, 6)), types: (tf.float64, tf.int64)>,
 'client_3': <BatchDataset shapes: ((None, 128, 9), (None, 6)), types: (tf.float64, tf.int64)>,
 'client_4': <BatchDataset shapes: ((None, 128, 9), (None, 6)), types: (tf.float64, tf.int64)>,
 'client_5': <BatchDataset shapes: ((None, 128, 9), (None, 6)), types: (tf.float64, tf.int64)>,
 'client_6': <BatchDataset shapes: ((None, 128, 9), (None, 6)), types: (tf.float64, tf.int64)>,
 'client_7': <BatchDataset shapes: ((None, 128, 9), (None, 6)), types: (tf.float64, tf.int64)>,
 'client_8': <BatchDataset shapes: ((None, 128, 9), (None, 6)), types: (tf.float64, tf.int64)>,
 'client_9': <BatchDataset shapes: ((None, 128, 9), (None, 6)), types: (tf.float64, tf.int64)>}

## Model

In [15]:
def create_keras_model():

    # Create keras model
    model = Sequential()

    # 1st convolution layer
    model.add(Input(shape=(128,9)))

    model.add(Conv1D(filters=32,kernel_size=2,strides=1,padding='same',activation=tf.nn.relu))
    model.add(MaxPool1D(pool_size=4,strides=2,padding='same'))

    model.add(Conv1D(filters=64,kernel_size=2,strides=1,padding='same',activation=tf.nn.relu))
    model.add(MaxPool1D(pool_size=4,strides=2,padding='same'))

    model.add(Conv1D(filters=128,kernel_size=2,strides=1,padding='same',activation=tf.nn.relu))
    model.add(MaxPool1D(pool_size=4,strides=2,padding='same'))
    
    # Fully connected layer
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(6, activation='softmax'))

    # Compile the model
    #model.compile(loss=SparseCategoricalCrossentropy(),
    #              optimizer=SGD(learning_rate=0.02),
    #              metrics=[SparseCategoricalAccuracy()])
    
    return model

# Summary model
keras_model = create_keras_model()
keras_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 128, 32)           608       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 64, 32)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 64, 64)            4160      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 32, 64)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 32, 128)           16512     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 16, 128)           0         
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0

In [16]:
lr = 0.01 
comms_round = 50
loss='categorical_crossentropy'
metrics = ['accuracy']
optimizer = SGD(learning_rate=lr, 
                decay=lr / comms_round, 
                momentum=0.9
               )      

## Federated Learning Setting

In [17]:
def weight_scalling_factor(clients_trn_data, client_name):
    client_names = list(clients_trn_data.keys())
    #get the bs
    bs = list(clients_trn_data[client_name])[0][0].shape[0]
    #first calculate the total training data points across clinets
    global_count = sum([tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy() for client_name in client_names])*bs
    # get the total number of data points held by a client
    local_count = tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy()*bs
    return local_count/global_count


def scale_model_weights(weight, scalar):
    '''function for scaling a models weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final



def sum_scaled_weights(scaled_weight_list):
    '''Return the sum of the listed scaled weights. The is equivalent to scaled avg of the weights'''
    avg_grad = list()
    #get the average grad accross all client gradients
    for grad_list_tuple in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)
        
    return avg_grad


def test_model(X_test, Y_test,  model, comm_round):
    cce = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    #logits = model.predict(X_test, batch_size=100)
    logits = model.predict(X_test)
    loss = cce(Y_test, logits)
    acc = accuracy_score(tf.argmax(logits, axis=1), tf.argmax(Y_test, axis=1))
    print('comm_round: {} | global_acc: {:.3%} | global_loss: {}'.format(comm_round, acc, loss))
    return acc, loss

In [18]:
#initialize global model
global_model = create_keras_model()
        
#commence global training loop
for comm_round in range(comms_round):
            
    # get the global model's weights - will serve as the initial weights for all local models
    global_weights = global_model.get_weights()
    
    #initial list to collect local model weights after scalling
    scaled_local_weight_list = list()

    #randomize client data - using keys
    client_names= list(clients_batched.keys())
    random.shuffle(client_names)
    
    #loop through each client and create new local model
    for client in client_names:
        local_model = create_keras_model()
        local_model.compile(loss=loss, 
                      optimizer=optimizer, 
                      metrics=metrics)
        
        #set local model weight to the weight of the global model
        local_model.set_weights(global_weights)
        
        #fit local model with client's data
        local_model.fit(clients_batched[client], epochs=1, verbose=0)
        
        #scale the model weights and add to list
        scaling_factor = weight_scalling_factor(clients_batched, client)
        scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
        scaled_local_weight_list.append(scaled_weights)
        
        #clear session to free memory after each communication round
        K.clear_session()
        
    #to get the average over all the local model, we simply take the sum of the scaled weights
    average_weights = sum_scaled_weights(scaled_local_weight_list)
    
    #update global model 
    global_model.set_weights(average_weights)

    #test global model and print out metrics after each communications round
    for(X_test, Y_test) in test_batched:
        global_acc, global_loss = test_model(X_test, Y_test, global_model, comm_round)

comm_round: 0 | global_acc: 59.688% | global_loss: 1.705342411994934
comm_round: 1 | global_acc: 71.157% | global_loss: 1.5064157247543335
comm_round: 2 | global_acc: 73.736% | global_loss: 1.4260972738265991
comm_round: 3 | global_acc: 77.367% | global_loss: 1.3703622817993164
comm_round: 4 | global_acc: 78.079% | global_loss: 1.352492094039917
comm_round: 5 | global_acc: 78.385% | global_loss: 1.321596384048462
comm_round: 6 | global_acc: 78.724% | global_loss: 1.3046586513519287
comm_round: 7 | global_acc: 80.794% | global_loss: 1.2948840856552124
comm_round: 8 | global_acc: 80.048% | global_loss: 1.2896400690078735
comm_round: 9 | global_acc: 79.912% | global_loss: 1.2874852418899536
comm_round: 10 | global_acc: 80.692% | global_loss: 1.2608797550201416
comm_round: 11 | global_acc: 82.525% | global_loss: 1.2512247562408447
comm_round: 12 | global_acc: 84.798% | global_loss: 1.2484595775604248
comm_round: 13 | global_acc: 84.730% | global_loss: 1.2345250844955444
comm_round: 14 | gl